In [0]:
# import os
# import re
# import cv2
# import pytesseract
# import numpy as np
# import pandas as pd
# import logging
# import json
# import math
# #from matplotlib import pyplot as plt   # Not displaying images
# from PIL import Image

# # -------------------------------
# # Logger Setup (INFO level)
# # -------------------------------
# logging.basicConfig(level=logging.INFO,
#                     format="%(asctime)s [%(levelname)s] %(message)s")
# logger = logging.getLogger("SectionExtractor")

# # -------------------------------
# # Utility Functions
# # -------------------------------
# def dbfs_to_local_path(dbfs_path):
#     """Convert a DBFS URI (e.g. "dbfs:/mnt/xxx") to a local path ("/dbfs/mnt/xxx")."""
#     if dbfs_path.startswith("dbfs:/"):
#         return "/dbfs/" + dbfs_path[len("dbfs:/"):]
#     return dbfs_path

# def sanitize_section_name(section):
#     """Convert section name into a safe file name."""
#     return section.lower().replace(" ", "_").replace("/", "_").replace(":", "")

# def safe_read_image_cv2(image_path):
#     local_path = dbfs_to_local_path(image_path)
#     logger.info(f"Reading image from: {local_path}")
#     if not os.path.exists(local_path):
#         raise FileNotFoundError(f"Image file not found at {local_path}")
#     img = cv2.imread(local_path)
#     if img is None:
#         raise ValueError(f"OpenCV failed to read image: {local_path}")
#     return img

# def safe_read_image_pil(image_path):
#     local_path = dbfs_to_local_path(image_path)
#     if not os.path.exists(local_path):
#         raise FileNotFoundError(f"Image file not found at {local_path}")
#     return Image.open(local_path)

# # -------------------------------
# # Preprocessing Functions
# # -------------------------------
# def preprocess_image(image, debug=False):
#     """Convert image to grayscale and apply adaptive thresholding."""
#     if len(image.shape) == 2:
#         gray = image
#     elif len(image.shape) == 3:
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     else:
#         raise ValueError("Unexpected number of channels in input image.")
#     thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY, 15, 9)
#     return thresh

# # -------------------------------
# # Generic OCR Functions
# # -------------------------------
# def detect_text_regions(thresh_img, debug=False):
#     contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     rois = []
#     for cnt in contours:
#         x, y, w, h = cv2.boundingRect(cnt)
#         if w > 30 and h > 15:
#             rois.append((x, y, w, h))
#     rois.sort(key=lambda b: (b[1], b[0]))
#     return rois

# def perform_ocr_on_rois(img, rois, debug=False):
#     results = []
#     for (x, y, w, h) in rois:
#         roi = img[y:y+h, x:x+w]
#         text = pytesseract.image_to_string(roi, config="--psm 6").strip() or "[BLANK]"
#         results.append((x, y, w, h, text))
#     return results

# # -------------------------------
# # DDR Pipeline
# # -------------------------------
# def process_daily_drilling_report(image_path, debug=False):
#     logger.info(f"Processing DDR image from: {image_path}")
#     img = safe_read_image_cv2(image_path)
#     coords = (1600, 0, 950, 185)  # adjust as needed
#     x, y, w, h = coords
#     cropped = img[y:y+h, x:x+w]
#     gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) if len(cropped.shape) != 2 else cropped
#     equalized = cv2.equalizeHist(gray)
#     blurred = cv2.GaussianBlur(equalized, (5,5), 0)
#     processed = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                        cv2.THRESH_BINARY, 11, 2)
#     ocr_text = pytesseract.image_to_string(processed, config="--psm 6").strip()
#     logger.info("DDR OCR extraction complete.")
#     expected_keys = ["Report Date", "Report Num", "Rig"]
#     combined = " ".join(ocr_text.splitlines()).strip()
#     combined = re.sub(r'\s+', ' ', combined)
#     extracted = {}
#     for i, key in enumerate(expected_keys):
#         pattern = (rf'{re.escape(key)}\s*:\s*(.*?)'
#                    rf'(?=\s*{re.escape(expected_keys[i+1])}\s*:|$)') if i < len(expected_keys)-1 else rf'{re.escape(key)}\s*:\s*(.*)'
#         match = re.search(pattern, combined, re.IGNORECASE)
#         extracted[key] = match.group(1).strip() if match and match.group(1).strip() else None
#     if extracted.get("Report Num"):
#         m = re.search(r'^(\d+)\.?$', extracted["Report Num"])
#         extracted["Report Num"] = m.group(1) if m else extracted["Report Num"]
#     df = pd.DataFrame(list(extracted.items()), columns=["Key", "Value"])
#     return {"DAILY DRILLING REPORT": extracted}, df

# # -------------------------------
# # WELL/JOB INFORMATION Pipeline (Real Code)
# # -------------------------------
# def process_well_job_info(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     # Use OCR on full image text
#     ocr_text = pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), config="--psm 6")
#     expected_keys = [
#         "Well Name", "Job Name", "Supervisor(s)", "Field", "Sec/Twn/Rng", "Phone",
#         "AFE #", "API #", "Email", "Contractor", "Elevation", "RKB",
#         "Spud Date", "Days from Spud", "Days on Loc", "MD/TVD", "24 Hr Footage",
#         "Present Operations", "Activity Planned"
#     ]
#     combined = " ".join(ocr_text.splitlines()).strip()
#     combined = re.sub(r'\s+', ' ', combined)
#     result = {}
#     for i, key in enumerate(expected_keys):
#         pattern = (rf'{re.escape(key)}\s*:\s*(.*?)'
#                    rf'(?=\s*{re.escape(expected_keys[i+1])}\s*:|$)') if i < len(expected_keys)-1 else rf'{re.escape(key)}\s*:\s*(.*)'
#         match = re.search(pattern, combined, re.IGNORECASE)
#         result[key] = match.group(1).strip() if match else ""
#     df = pd.DataFrame(list(result.items()), columns=["Key", "Value"])
#     return {"WELL/JOB INFORMATION": result}, df

# # -------------------------------
# # MUD Pipeline (Real Code)
# # -------------------------------
# def process_mud(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh_img = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh_img, debug=debug)
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     expected_headers = [
#         "Type", "Weight In", "Weight Out", "pH", "CAKE",
#         "GELS (10s/10m/30m)", "Oil/Water", "FV", "ES", "PV",
#         "YP", "CL", "Ca", "LGS", "WL", "HTHP Loss", "3 RPM",
#         "6 RPM", "Mud Pits and Hole Volume", "24 Hr Loss",
#         "Total Loss", "Comments"
#     ]
#     def parse_value_row_tokens(expected_headers, tokens):
#         expected_token_count = (len(expected_headers) - 1) + 3
#         if len(tokens) < expected_token_count:
#             tokens += ["[BLANK]"] * (expected_token_count - len(tokens))
#         elif len(tokens) > expected_token_count:
#             tokens = tokens[:expected_token_count]
#         result = {}
#         idx = 0
#         for header in expected_headers:
#             if header == "GELS (10s/10m/30m)":
#                 gels_tokens = tokens[idx:idx+3]
#                 result[header] = {"10s": gels_tokens[0], "10m": gels_tokens[1], "30m": gels_tokens[2]}
#                 idx += 3
#             else:
#                 result[header] = tokens[idx]
#                 idx += 1
#         return result
#     def build_mud_dict_from_rois(roi_texts, expected_headers):
#         row_tolerance = 10
#         rows = []
#         current_row = []
#         prev_y = None
#         for (x, y, w, h, text) in roi_texts:
#             current_row.append((x, y, w, h, text))
#             prev_y = y
#         rows = [current_row]  # For simplicity, assume one grouped row.
#         if not rows:
#             logger.error("No rows grouped for MUD.")
#             return {}
#         tokens = rows[0][0][4].split()  # Simplified; replace with your grouping logic.
#         return parse_value_row_tokens(expected_headers, tokens)
#     mud_dict = build_mud_dict_from_rois(roi_texts, expected_headers)
#     df = pd.DataFrame(list(mud_dict.items()), columns=["Key", "Value"]) if isinstance(mud_dict, dict) else pd.DataFrame(mud_dict)
#     return {"MUD": mud_dict}, df

# # -------------------------------
# # SURVEY DATA Pipeline (Real Code)
# # -------------------------------
# def build_survey_dict_from_rois(roi_texts, expected_headers):
#     row_tolerance = 10
#     rows = []
#     current_row = []
#     prev_y = None
#     for (x, y, w, h, text) in roi_texts:
#         if prev_y is None or abs(y - prev_y) <= row_tolerance:
#             current_row.append((x, y, w, h, text))
#         else:
#             rows.append(current_row)
#             current_row = [(x, y, w, h, text)]
#         prev_y = y
#     if current_row:
#         rows.append(current_row)
    
#     row_strings = []
#     for i, row in enumerate(rows):
#         row.sort(key=lambda cell: cell[0])
#         line = " ".join(cell[4] for cell in row)
#         row_strings.append(line)
#         logger.info(f"Grouped Row {i}: {line}")
    
#     all_lines = []
#     for line in row_strings:
#         for subline in line.split("\n"):
#             subline = subline.strip()
#             if subline:
#                 all_lines.append(subline)
#     logger.info(f"All extracted lines: {all_lines}")
    
#     data_lines = []
#     for line in all_lines:
#         tokens = re.split(r'\s{2,}', line)
#         if len(tokens) == 1:
#             tokens = line.split()
#         lower_tokens = [t.lower() for t in tokens]
#         if "md" in lower_tokens and "inclination" in lower_tokens:
#             logger.info(f"Skipping header line: {tokens}")
#             continue
#         if len(tokens) < len(expected_headers):
#             logger.warning(f"Line has fewer tokens than expected: {tokens}")
#             continue
#         tokens = tokens[:len(expected_headers)]
#         data_lines.append(tokens)
    
#     survey_list = []
#     for tokens in data_lines:
#         row_dict = {expected_headers[i]: tokens[i] for i in range(len(expected_headers))}
#         survey_list.append(row_dict)
#     return survey_list

# def sort_survey_data(survey_list):
#     def md_value(row):
#         try:
#             return float(row["MD"].replace(",", ""))
#         except Exception:
#             return 0
#     sorted_list = sorted(survey_list, key=md_value, reverse=False)
#     filtered_list = [row for row in sorted_list if not row["MD"].upper().startswith("SURVEY")]
#     return filtered_list

# def process_survey(survey_img_path, debug=True):
#     expected_headers = ["MD", "Inclination", "Azimuth", "DLS", "TVD"]
#     img = safe_read_image_cv2(survey_img_path)
#     # if debug:
#         # show_image("Original Survey Image", img, size=(12,12))
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.adaptiveThreshold(
#         gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#         cv2.THRESH_BINARY, 15, 9
#     )
#     # if debug:
#         # show_image("Adaptive Threshold", thresh, cmap="gray")
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     rois = []
#     for cnt in contours:
#         x, y, w, h = cv2.boundingRect(cnt)
#         if w > 30 and h > 15:
#             rois.append((x, y, w, h))
#     rois.sort(key=lambda b: (b[1], b[0]))
#     if debug:
#         debug_img = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
#         for (x, y, w, h) in rois:
#             cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0,255,0), 2)
#         show_image("Detected Text Regions", debug_img)
#     roi_texts = []
#     for (x, y, w, h) in rois:
#         roi = img[y:y+h, x:x+w]
#         text = pytesseract.image_to_string(roi, config="--psm 6").strip()
#         if not text:
#             text = "[BLANK]"
#         roi_texts.append((x, y, w, h, text))
#         if debug:
#             logger.info(f"OCR Box ({x},{y},{w},{h}): {text}")
#     survey_list = build_survey_dict_from_rois(roi_texts, expected_headers)
#     survey_list = sort_survey_data(survey_list)
#     final_output = {"SURVEY DATA": survey_list}
#     df = pd.DataFrame(survey_list)
#     return final_output, df

# # -------------------------------
# # DIR INFO Pipeline (Real Code)
# # -------------------------------
# def process_dir_info(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh, debug=debug)
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     def build_dir_info_dict(roi_texts):
#         all_texts = [t[4] for t in roi_texts]
#         daily_cum_idx = next((i for i, txt in enumerate(all_texts)
#                                if "daily" in txt.lower() and "cumulative" in txt.lower()), None)
#         if daily_cum_idx is None:
#             logger.warning("Could not find 'Daily Cumulative' bounding box.")
#             return {}, pd.DataFrame()
#         cat_idx = daily_cum_idx + 1
#         if cat_idx >= len(all_texts):
#             logger.warning("No bounding box after 'Daily Cumulative'.")
#             return {}, pd.DataFrame()
#         categories_box = all_texts[cat_idx]
#         lines = [ln.strip() for ln in categories_box.split("\n") if ln.strip()]
#         if len(lines) < 5:
#             logger.warning(f"Expected 5 category lines, got {len(lines)}: {lines}")
#         def safe_get(idx):
#             return all_texts[idx] if 0 <= idx < len(all_texts) else ""
#         structured = []
#         for i in range(4):
#             cat_name = lines[i] if i < len(lines) else f"Unknown Category {i+1}"
#             daily_box = safe_get(cat_idx + 1 + (i * 2))
#             cum_box = safe_get(cat_idx + 2 + (i * 2))
#             structured.append({
#                 "Category": cat_name,
#                 "Daily": "" if daily_box == "[BLANK]" else daily_box,
#                 "Cumulative": "" if cum_box == "[BLANK]" else cum_box
#             })
#         last_box = safe_get(cat_idx + 9)
#         last_cat = lines[4] if len(lines) >= 5 else "Rotating Footage"
#         remainder = last_box.replace(last_cat, "").strip()
#         tokens = remainder.split()
#         daily_val = tokens[0] if len(tokens) >= 2 else ""
#         cum_val = tokens[1] if len(tokens) >= 2 else ""
#         structured.append({
#             "Category": last_cat,
#             "Daily": "" if daily_val == "[BLANK]" else daily_val,
#             "Cumulative": "" if cum_val == "[BLANK]" else cum_val
#         })
#         df = pd.DataFrame(structured)
#         return {"DIR INFO": structured}, df
#     return build_dir_info_dict(roi_texts)

# # -------------------------------
# # DRILL BITS Pipeline (Real Code)
# # -------------------------------
# def process_drill_bits(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh_img = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh_img, debug=debug)
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     bit_info_list, df = build_bit_info_dict_from_rois(roi_texts, debug=debug)
#     final_output = {"BIT DETAILS": bit_info_list}
#     with open("bit_info_data.json", "w") as f:
#         json.dump(final_output, f, indent=4)
#     df.to_csv("bit_info_data.csv", index=False)
#     return {"DRILL BITS": final_output}, df

# # -------------------------------
# # CASING Pipeline (Real Code)
# # -------------------------------
# def build_casing_dict_from_rois(roi_texts, expected_headers, debug=False):
#     row_tolerance = 10
#     rows = []
#     current_row = []
#     prev_y = None
#     for (x, y, w, h, text) in roi_texts:
#         tokens = text.split()
#         if tokens and tokens[0].lower() in ["og", "casing"]:
#             continue
#         if prev_y is None or abs(y - prev_y) <= row_tolerance:
#             current_row.append((x, y, w, h, text))
#         else:
#             rows.append(current_row)
#             current_row = [(x, y, w, h, text)]
#         prev_y = y
#     if current_row:
#         rows.append(current_row)
#     row_strings = []
#     for i, row in enumerate(rows):
#         row.sort(key=lambda cell: cell[0])
#         line = " ".join(cell[4] for cell in row).strip()
#         row_strings.append(line)
#         logger.debug(f"Casing Row {i}: {line}")
#     all_lines = []
#     for line in row_strings:
#         for sub in line.split("\n"):
#             sub = sub.strip()
#             if sub:
#                 all_lines.append(sub)
#     data_lines = []
#     for line in all_lines:
#         tokens = re.split(r'\s{2,}', line)
#         if len(tokens) == 1:
#             tokens = line.split()
#         lower_tokens = [t.lower() for t in tokens]
#         if "type" in lower_tokens and "size" in lower_tokens:
#             continue
#         if len(tokens) < len(expected_headers):
#             continue
#         tokens = tokens[:len(expected_headers)]
#         data_lines.append(tokens)
#     casing_list = [{expected_headers[i]: tokens[i] for i in range(len(expected_headers))}
#                    for tokens in data_lines]
#     logger.debug(f"Final Casing List: {casing_list}")
#     return casing_list

# def process_casing(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh, debug=debug)
#     logger.info(f"Detected {len(rois)} text regions for CASING")
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     expected_headers = ["Type", "Size", "Weight", "Grade", "Connection", "Top MD", "Bottom MD", "TOC"]
#     casing_list = build_casing_dict_from_rois(roi_texts, expected_headers, debug=debug)
#     df = pd.DataFrame(casing_list)
#     logger.info(f"CASING DataFrame shape: {df.shape}")
#     return {"CASING": casing_list}, df

# # -------------------------------
# # BOP Pipeline (Real Code)
# # -------------------------------
# def process_bop(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     ocr_text = pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), config="--psm 6").strip()
#     logger.info("BOP OCR extraction complete.")
#     patterns = {
#         "Last BOP Test Date": r"Last BOP Test Date\s*:\s*(\d{1,2}/\d{1,2}/\d{2,4})",
#         "Last BOP Drill": r"Last BOP Drill\s*:\s*(\d{1,2}/\d{1,2}/\d{2,4})",
#         "Next BOP Test": r"Next BOP Test\s*:\s*(\d{1,2}/\d{1,2}/\d{2,4})"
#     }
#     bop_info = {}
#     for key, pattern in patterns.items():
#         match = re.search(pattern, ocr_text, re.IGNORECASE)
#         bop_info[key] = match.group(1) if match else ""
#     df = pd.DataFrame(list(bop_info.items()), columns=["Key", "Value"])
#     logger.info(f"BOP DataFrame shape: {df.shape}")
#     return {"BOP": bop_info}, df

# # -------------------------------
# # PERSONNEL Pipeline (Real Code)
# # -------------------------------
# def detect_text_regions_personnel(thresh_img, debug=False):
#     contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     rois = []
#     for cnt in contours:
#         x, y, w, h = cv2.boundingRect(cnt)
#         if w > 30 and h > 15:
#             rois.append((x, y, w, h))
#     rois.sort(key=lambda b: (b[1], b[0]))
#     logger.debug(f"Detected {len(rois)} text regions for personnel.")
#     return rois

# def perform_ocr_on_rois_personnel(img, rois, debug=False):
#     results = []
#     for (x, y, w, h) in rois:
#         roi = img[y:y+h, x:x+w]
#         text = pytesseract.image_to_string(roi, config="--psm 6").strip() or "[BLANK]"
#         results.append((x, y, w, h, text))
#         logger.debug(f"Personnel ROI ({x},{y},{w},{h}): {text}")
#     return results

# def group_rois_by_row(roi_results, y_threshold=20):
#     roi_with_center = [(x, y, w, h, text, y + h/2) for (x, y, w, h, text) in roi_results]
#     roi_with_center.sort(key=lambda r: r[5])
#     groups = []
#     current_group = []
#     current_y = None
#     for roi in roi_with_center:
#         x, y, w, h, text, y_center = roi
#         if current_y is None or abs(y_center - current_y) <= y_threshold:
#             current_group.append((x, y, w, h, text))
#             current_y = y_center if current_y is None else (current_y + y_center) / 2
#         else:
#             groups.append(current_group)
#             current_group = [(x, y, w, h, text)]
#             current_y = y_center
#     if current_group:
#         groups.append(current_group)
#     return groups

# def preprocess_personnel_data_from_rows(groups):
#     personnel_data = []
#     for group in groups:
#         group.sort(key=lambda r: r[0])
#         row_text = " ".join([r[4] for r in group]).strip()
#         if any(kw in row_text.upper() for kw in ["PERSONNEL", "COMPANY", "CONTRACTOR"]):
#             continue
#         tokens = row_text.split()
#         numeric_tokens = re.findall(r'\d+(?:\.\d+)?', row_text)
#         if len(numeric_tokens) < 3:
#             continue
#         try:
#             no_personnel = int(float(numeric_tokens[-3]))
#             daily_hours = int(float(numeric_tokens[-2]))
#             cumulative_hours = int(float(numeric_tokens[-1]))
#         except Exception as e:
#             logger.error(f"Error parsing numbers from row: {row_text} => {e}")
#             continue
#         # Remove the numeric tokens from text to extract company name
#         company = row_text
#         for num in numeric_tokens[-3:]:
#             company = company.replace(num, "")
#         company = company.strip()
#         row_dict = {
#             "Company": company,
#             "Contractor": "Service Company",
#             "No. Personnel": no_personnel,
#             "Daily Hours": daily_hours,
#             "Cumulative Hours": cumulative_hours
#         }
#         logger.info(f"Parsed personnel row: {row_dict}")
#         personnel_data.append(row_dict)
#     return {"PERSONNEL": personnel_data}

# def process_personnel(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     thresh_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                        cv2.THRESH_BINARY, 11, 2)
#     rois = detect_text_regions_personnel(thresh_img, debug=debug)
#     roi_results = perform_ocr_on_rois_personnel(img, rois, debug=debug)
#     groups = group_rois_by_row(roi_results, y_threshold=20)
#     data_dict = preprocess_personnel_data_from_rows(groups)
#     df = pd.DataFrame(data_dict["PERSONNEL"]) if data_dict["PERSONNEL"] else pd.DataFrame(
#         columns=["Company", "Contractor", "No. Personnel", "Daily Hours", "Cumulative Hours"])
#     return data_dict, df

# # -------------------------------
# # DAILY NUMBERS: OBSERVATION & INTERVENTION Pipeline (Real Code)
# # -------------------------------
# def process_obs_int(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh, debug=debug)
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     header_str = "daily numbers: observation & intervention"
#     types_list, numbers_list = [], []
#     for (_, _, _, _, text) in roi_texts:
#         clean = text.strip()
#         if clean.lower() in [header_str, "number", "[blank]"]:
#             continue
#         try:
#             float(clean)
#             numbers_list.append(clean)
#         except ValueError:
#             types_list.append(clean)
#     # Use the maximum length found; do not force 5 if not present.
#     count = max(len(types_list), len(numbers_list))
#     while len(types_list) < count:
#         types_list.append("")
#     while len(numbers_list) < count:
#         numbers_list.append("")
#     structured = [{"Type": types_list[i], "Number": numbers_list[i]} for i in range(count)]
#     df = pd.DataFrame(structured)
#     logger.info(f"DAILY NUMBERS: OBSERVATION & INTERVENTION DataFrame shape: {df.shape}")
#     return {"DAILY NUMBERS: OBSERVATION & INTERVENTION": structured}, df

# # -------------------------------
# # BHA Pipeline (Real Code)
# # -------------------------------
# def extract_bha_data(image_path):
#     image = safe_read_image_cv2(image_path)
#     ocr_text = pytesseract.image_to_string(image)
#     patterns = {
#         "Drill Pipe Detail": r"Drill Pipe Detail:\s*([^\n]+)",
#         "Size": r"Size:\s*([\d.]+)\b",
#         "Wt./Ft": r"Wt\./Ft:\s*([\d.]+)\b",
#         "Connection": r"Connection:\s*([\w\d-]+)\b",
#         "ID": r"ID:\s*([\d.]+)\b",
#         "Drill Bit": r"Drill Bit:\s*([^\n;]+)",
#         "Motor": r"Motor:\s*([^\n;]+)",
#         "MWD Tool": r"MWD Tool:\s*([^\n;]+)",
#         "Monel Collar": r"Monel Collar:\s*([^\n;]+)",
#         "X-Over": r"X-Over:\s*([^\n;]+)",
#         "Sub": r"Sub:\s*([^\n;]+)",
#         "HWDP": r"HWDP:\s*([^\n;]+)",
#         "Drill Pipe": r"Drill Pipe:\s*([\d.]+(?:\" DP)?)",
#         "Reamer": r"Reamer:\s*([^\n;]+)",
#         "Shock Sub": r"Shock Sub:\s*([^\n;]+)",
#         "Total Length": r"Total Length:\s*(\d+)\b"
#     }
#     bha_data = {}
#     for key, pattern in patterns.items():
#         match = re.search(pattern, ocr_text)
#         if match:
#             bha_data[key] = match.group(1).strip()
#     if "Drill Pipe Detail" in bha_data:
#         detail = bha_data["Drill Pipe Detail"]
#         for remove_key in ["Size", "Wt./Ft", "Connection", "ID"]:
#             if remove_key in bha_data:
#                 detail = re.sub(rf"{remove_key}:\s*{re.escape(bha_data[remove_key])}", "", detail).strip(",; ")
#         bha_data["Drill Pipe Detail"] = detail
#     structured_data = {
#         "BHA": {
#             "Drill Pipe Detail": bha_data.get("Drill Pipe Detail", ""),
#             "Size": bha_data.get("Size", ""),
#             "Wt./Ft": bha_data.get("Wt./Ft", ""),
#             "Connection": bha_data.get("Connection", ""),
#             "ID": bha_data.get("ID", ""),
#             "BHA #4": {
#                 "Drill Bit": bha_data.get("Drill Bit", ""),
#                 "Motor": bha_data.get("Motor", ""),
#                 "MWD Tool": bha_data.get("MWD Tool", ""),
#                 "Monel Collar": bha_data.get("Monel Collar", ""),
#                 "X-Over": bha_data.get("X-Over", ""),
#                 "Sub": bha_data.get("Sub", ""),
#                 "HWDP": bha_data.get("HWDP", ""),
#                 "Drill Pipe": bha_data.get("Drill Pipe", ""),
#                 "Reamer": bha_data.get("Reamer", ""),
#                 "Shock Sub": bha_data.get("Shock Sub", "")
#             },
#             "Total Length": bha_data.get("Total Length", "")
#         }
#     }
#     return structured_data

# def process_bha(image_path, debug=False):
#     bha_data = extract_bha_data(image_path)
#     df = pd.DataFrame([bha_data.get("BHA", {})])
#     return {"BHA": bha_data.get("BHA", {})}, df

# # -------------------------------
# # PUMPS Pipeline (Real Code)
# # -------------------------------
# def parse_pumps_table(ocr_text):
#     pump_pattern = re.compile(
#         r"^(\d+)?\s*"               # Number (optional)
#         r"(BOMCO)\s+(TRIPLEX)\s+"    # Model, Type
#         r"(\d+)?\s*"                # HHP (optional)
#         r"(\d+)\s+"                 # Efficiency
#         r"([\d.]+)\s+"              # Stroke\(in\)
#         r"([\d.]+)\s+"              # Liner\(in\)
#         r"(\d+)\s+"                 # P-Rating\(psi\)
#         r"(\d+)\s+"                 # P-Limit\(psi\)
#         r"(\d+)\s+"                 # SPM Rating
#         r"(\d+)\s*$",               # SPM Limit
#         re.IGNORECASE
#     )
#     pumps = []
#     lines = ocr_text.splitlines()
#     for line in lines:
#         line = line.strip()
#         match = pump_pattern.match(line)
#         if match:
#             number, model, pump_type, hhp, efficiency, stroke, liner, p_rating, p_limit, spm_rating, spm_limit = match.groups()
#             pumps.append({
#                 "Number": number if number else "",
#                 "Model": model,
#                 "Type": pump_type,
#                 "HHP": hhp if hhp else "",
#                 "Efficiency": efficiency,
#                 "Stroke(in)": stroke,
#                 "Liner(in)": liner,
#                 "P-Rating(psi)": p_rating,
#                 "P-Limit(psi)": p_limit,
#                 "SPM Rating": spm_rating,
#                 "SPM Limit": spm_limit
#             })
#     return pumps

# def parse_drilling_circ_rates(ocr_text):
#     circ_rates = []
#     segments = re.split(r"(?=Drilling/Circ Rate \d+)", ocr_text)
#     pattern = re.compile(
#         r"Drilling/Circ Rate\s+(\d+).*?"       
#         r"(\d+)\s+PS[!I].*?"                   
#         r"@\s*(\d+).*?"                        
#         r"([\d.]+)\s+Gal/Stoke.*?"              
#         r"([\d.]+)\s+GPM.*?"                    
#         r"([\d.]+)\s+BPM.*?"                    
#         r"([\d.]+)\s+DC.*?"                     
#         r"([\d.]+)\s+DP",                      
#         re.IGNORECASE | re.DOTALL
#     )
#     for seg in segments:
#         seg = seg.strip()
#         if not seg.startswith("Drilling/Circ Rate"):
#             continue
#         seg_clean = " ".join(seg.splitlines())
#         match = pattern.search(seg_clean)
#         if match:
#             rate_id, pressure, spm, gal_stroke, gpm, bpm, dc, dp = match.groups()
#             circ_rates.append({
#                 "RateID": rate_id,
#                 "Pressure(PSI)": pressure,
#                 "SPM": spm,
#                 "Gal/Stoke": gal_stroke,
#                 "GPM": gpm,
#                 "BPM": bpm,
#                 "DC": dc,
#                 "DP": dp
#             })
#         else:
#             logger.warning(f"No match found in segment:\n{seg_clean}")
#     return circ_rates

# def process_pumps(image_path, debug=False):
#     pil_img = safe_read_image_cv2(image_path)
#     ocr_text = pytesseract.image_to_string(pil_img, config="--psm 6").strip()
#     logger.info("Pumps OCR Text:\n" + ocr_text)
#     pumps = parse_pumps_table(ocr_text)
#     circ_rates = parse_drilling_circ_rates(ocr_text)
#     final_data = {"Pumps": pumps, "DrillingCircRates": circ_rates}
#     df_pumps = pd.DataFrame(pumps)
#     df_circ = pd.DataFrame(circ_rates)
#     if not df_pumps.empty and not df_circ.empty:
#         df = pd.concat([df_pumps, df_circ], axis=0, ignore_index=True)
#     elif not df_pumps.empty:
#         df = df_pumps
#     else:
#         df = df_circ
#     return {"PUMPS": final_data}, df

# # -------------------------------
# # COST DATA Pipeline (Real Code)
# # -------------------------------
# def process_cost_data(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     ocr_text = pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), config="--psm 6")
#     logger.info("Cost OCR extraction complete.")
#     expected_keys = [
#         "Drilling AFE Amount", "Daily Drilling Cost", "Cumulative Drilling Cost",
#         "Cumulative Well Cost", "Daily Mud Cost", "Cumulative Mud Cost"
#     ]
#     combined = " ".join(ocr_text.splitlines()).strip()
#     combined = re.sub(r'\s+', ' ', combined)
#     extracted = {}
#     for i, key in enumerate(expected_keys):
#         pattern = (rf'{re.escape(key)}\s*:\s*(.*?)(?=\s*{re.escape(expected_keys[i+1])}\s*:|$)') if i < len(expected_keys)-1 else rf'{re.escape(key)}\s*:\s*(.*)'
#         match = re.search(pattern, combined, re.IGNORECASE)
#         extracted[key] = match.group(1).strip() if match and match.group(1).strip() else ""
#     df = pd.DataFrame(list(extracted.items()), columns=["Key", "Value"])
#     logger.info(f"COST DataFrame shape: {df.shape}")
#     return {"COST DATA": extracted}, df

# # -------------------------------
# # TIME BREAKDOWN Pipeline (Real Code)
# # -------------------------------
# def detect_text_regions_tb(thresh_img, debug=False):
#     contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     rois = []
#     for cnt in contours:
#         x, y, w, h = cv2.boundingRect(cnt)
#         if w > 30 and h > 15:
#             rois.append((x, y, w, h))
#     rois.sort(key=lambda b: (b[1], b[0]))
#     return rois

# def perform_ocr_on_rois_tb(img, rois, debug=False):
#     results = []
#     for (x, y, w, h) in rois:
#         roi = img[y:y+h, x:x+w]
#         text = pytesseract.image_to_string(roi, config="--psm 6").strip() or "[BLANK]"
#         results.append((x, y, w, h, text))
#         logger.debug(f"Time Breakdown OCR Box ({x},{y},{w},{h}): {text}")
#     return results

# def group_ocr_rows(roi_results, y_threshold=20):
#     groups = []
#     roi_results_sorted = sorted(roi_results, key=lambda r: r[1])
#     current_group = []
#     current_y = None
#     for roi in roi_results_sorted:
#         x, y, w, h, text = roi
#         if current_y is None or abs(y - current_y) <= y_threshold:
#             current_group.append(roi)
#             current_y = y if current_y is None else (current_y + y) / 2
#         else:
#             groups.append(current_group)
#             current_group = [roi]
#             current_y = y
#     if current_group:
#         groups.append(current_group)
#     return groups

# def parse_operations_description(ops_text):
#     ops_data = {"Observations": [], "Targets": []}
#     m = re.search(r"MWD Tool", ops_text, re.IGNORECASE)
#     if m:
#         ops_data["Observations"].append("Sample observation")
#     else:
#         ops_data["Observations"].append("No observation")
#     return ops_data

# def parse_row_text(row_text):
#     clean_text = " ".join(row_text.split())
#     if not re.match(r"\d{2}:\d{2}", clean_text):
#         return None
#     tokens = clean_text.split()
#     if len(tokens) < 8:
#         return None
#     phase_activity = " ".join(tokens[5:])
#     m = re.search(r"^(?P<phase>.+?)\s+(?P<activity>DR[-]?Drilling)\s+(?P<ops>.*)$", phase_activity, re.IGNORECASE)
#     if m:
#         phase = m.group("phase")
#         activity = m.group("activity")
#         ops = m.group("ops")
#     else:
#         phase = tokens[5]
#         activity = tokens[6] if len(tokens) > 6 else ""
#         ops = " ".join(tokens[7:]) if len(tokens) > 7 else ""
#     return {
#         "From": tokens[0],
#         "To": tokens[1],
#         "Hours": tokens[2],
#         "Depth Start": tokens[3],
#         "Depth End": tokens[4],
#         "Phase": phase,
#         "Activity": activity,
#         "Operations Description": parse_operations_description(ops)
#     }

# def parse_all_rows_from_text(full_text):
#     row_chunks = re.split(r"(?=\d{2}:\d{2}\s+\d{2}:\d{2})", full_text)
#     rows = []
#     for chunk in row_chunks:
#         chunk = chunk.strip()
#         parsed = parse_row_text(chunk)
#         if parsed:
#             rows.append(parsed)
#     return rows

# def parse_all_rows_from_ocr_groups(roi_results):
#     rows = []
#     groups = group_ocr_rows(roi_results, y_threshold=20)
#     for group in groups:
#         group_sorted = sorted(group, key=lambda r: r[0])
#         row_text = " ".join([text for (x, y, w, h, text) in group_sorted])
#         parsed = parse_row_text(row_text)
#         if parsed:
#             rows.append(parsed)
#     return rows

# def process_time_breakdown(image_paths, debug=False):
#     rows_all = []
#     for img_path in image_paths:
#         img = safe_read_image_cv2(img_path)
#         thresh = preprocess_image(img, debug=debug)
#         rois = detect_text_regions_tb(thresh, debug=debug)
#         ocr_results = perform_ocr_on_rois_tb(img, rois, debug=debug)
#         rows = parse_all_rows_from_ocr_groups(ocr_results)
#         if not rows:
#             full_text = pytesseract.image_to_string(thresh, config="--psm 6")
#             rows = parse_all_rows_from_text(full_text)
#         rows_all.extend(rows)
#     final_output = {"TIME BREAKDOWN": rows_all}
#     df = pd.json_normalize(rows_all) if rows_all else pd.DataFrame()
#     return final_output, df

# # -------------------------------
# # CONSUMABLES Pipeline (Real Code)
# # -------------------------------
# def build_consumables_dict_from_rois(roi_texts, debug=False):
#     row_tolerance = 10
#     rows = []
#     current_row = []
#     prev_y = None
#     for (x, y, w, h, text) in roi_texts:
#         if prev_y is None or abs(y - prev_y) <= row_tolerance:
#             current_row.append((x, y, w, h, text))
#         else:
#             rows.append(current_row)
#             current_row = [(x, y, w, h, text)]
#         prev_y = y
#     if current_row:
#         rows.append(current_row)
#     grouped_rows = []
#     for i, row in enumerate(rows):
#         row.sort(key=lambda cell: cell[0])
#         line = " ".join(cell[4] for cell in row).strip()
#         grouped_rows.append(line)
#         logger.info(f"Consumables Row {i}: {line}")
#     data_rows = []
#     for line in grouped_rows:
#         lower_line = line.lower()
#         if ("consumable" in lower_line and "received" in lower_line) or "nun" in lower_line:
#             continue
#         if len(line.split()) < 5:
#             continue
#         data_rows.append(line)
#     consumables_list = []
#     for line in data_rows:
#         tokens = re.split(r'\s+', line)
#         if len(tokens) > 5:
#             first = " ".join(tokens[:-4])
#             tokens = [first] + tokens[-4:]
#         if len(tokens) != 5:
#             logger.warning(f"Skipping row (unexpected token count): {tokens}")
#             continue
#         row_dict = {
#             "Consumable": tokens[0],
#             "Daily Received (gal)": tokens[1],
#             "Daily Used (gal)": tokens[2],
#             "Cumulative Used (gal)": tokens[3],
#             "Daily on Hand (gal)": tokens[4]
#         }
#         consumables_list.append(row_dict)
#     return consumables_list

# def process_consumables(image_path, debug=False):
#     img = safe_read_image_cv2(image_path)
#     thresh = preprocess_image(img, debug=debug)
#     rois = detect_text_regions(thresh, debug=debug)
#     logger.info(f"Detected {len(rois)} text regions for CONSUMABLES")
#     roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
#     consumables_list = build_consumables_dict_from_rois(roi_texts, debug=debug)
#     df = pd.DataFrame(consumables_list)
#     logger.info(f"CONSUMABLES DataFrame shape: {df.shape}")
#     return {"CONSUMABLES": consumables_list}, df

# # -------------------------------
# # Aggregator Main Process
# # -------------------------------
# def main():
#     debug = False  # Set to False to reduce verbosity
#     image_paths = {
#         "DAILY DRILLING REPORT": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png",
#         "WELL/JOB INFORMATION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_2.png",
#         "MUD": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_3.png",
#         "SURVEY DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_4.png",
#         "DIR INFO": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_5.png",
#         "DRILL BITS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_6.png",
#         "CASING": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_7.png",
#         "BOP": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_8.png",
#         "PERSONNEL": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_9.png",
#         "DAILY NUMBERS: OBSERVATION & INTERVENTION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_10.png",
#         "BHA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_11.png",
#         "PUMPS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_12.png",
#         "COST DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_13.png",
#         "TIME BREAKDOWN": ["dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png",
#                            "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"],
#         "CONSUMABLES": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"
#     }
#     output_folder = dbfs_to_local_path("dbfs:/mnt/mini-proj-dd/final_results")
#     os.makedirs(output_folder, exist_ok=True)
    
#     aggregated_json = {}
#     aggregated_df = pd.DataFrame()
    
#     processes = [
#         ("DAILY DRILLING REPORT", process_daily_drilling_report, image_paths.get("DAILY DRILLING REPORT")),
#         ("WELL/JOB INFORMATION", process_well_job_info, image_paths.get("WELL/JOB INFORMATION")),
#         ("MUD", process_mud, image_paths.get("MUD")),
#         ("SURVEY DATA", process_survey, image_paths.get("SURVEY DATA")),
#         ("DIR INFO", process_dir_info, image_paths.get("DIR INFO")),
#         ("DRILL BITS", process_drill_bits, image_paths.get("DRILL BITS")),
#         ("CASING", process_casing, image_paths.get("CASING")),
#         ("BOP", process_bop, image_paths.get("BOP")),
#         ("PERSONNEL", process_personnel, image_paths.get("PERSONNEL")),
#         ("DAILY NUMBERS: OBSERVATION & INTERVENTION", process_obs_int, image_paths.get("DAILY NUMBERS: OBSERVATION & INTERVENTION")),
#         ("BHA", process_bha, image_paths.get("BHA")),
#         ("PUMPS", process_pumps, image_paths.get("PUMPS")),
#         ("COST DATA", process_cost_data, image_paths.get("COST DATA")),
#         ("TIME BREAKDOWN", lambda paths, d: process_time_breakdown(paths, d), image_paths.get("TIME BREAKDOWN")),
#         ("CONSUMABLES", process_consumables, image_paths.get("CONSUMABLES"))
#     ]
    
#     for section, func, img_path in processes:
#         try:
#             logger.info(f"Processing section: {section}")
#             data_json, df = func(img_path, debug)
#             safe_section = sanitize_section_name(section)
#             aggregated_json[section] = data_json.get(section, data_json)
#             aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)
#             logger.info(f"{section} output processed.")
#             section_json_file = os.path.join(output_folder, f"{safe_section}.json")
#             with open(section_json_file, "w") as f:
#                 json.dump(data_json, f, indent=4)
#             section_csv_file = os.path.join(output_folder, f"{safe_section}.csv")
#             df.to_csv(section_csv_file, index=False)
#             logger.info(f"Saved {section} outputs to JSON and CSV.")
#         except Exception as e:
#             logger.error(f"{section} processing failed: {e}")
    
#     agg_json_path = os.path.join(output_folder, "aggregated_data.json")
#     with open(agg_json_path, "w") as f:
#         json.dump(aggregated_json, f, indent=4)
#     agg_csv_path = os.path.join(output_folder, "aggregated_data.csv")
#     aggregated_df.to_csv(agg_csv_path, index=False)
#     logger.info(f"Aggregated outputs saved to {agg_json_path} and {agg_csv_path}.")
#     print("----- Aggregated JSON Output -----")
#     print(json.dumps(aggregated_json, indent=4))

# if __name__ == "__main__":
#     main()


ERROR:SectionExtractor:DRILL BITS processing failed: name 'build_bit_info_dict_from_rois' is not defined
Drilling/Circ Rate 1  Driling/Circ Rate 2


----- Aggregated JSON Output -----
{
    "DAILY DRILLING REPORT": {
        "Report Date": "7/4/2024",
        "Report Num": "11",
        "Rig": "Cyclone 39"
    },
    "WELL/JOB INFORMATION": {
        "Well Name": "Ross Fee 4371-31-7-15 MH",
        "Job Name": "Drilling",
        "Supervisor(s)": "CHAD MILLER / ED COOLEY",
        "Field": "XBE",
        "Sec/Twn/Rng": "31, 43N, 71W",
        "Phone": "307-315-1908",
        "AFE #": "240098",
        "API #": "49-005-78911",
        "Email": "cyclone39@aec-denver.com",
        "Contractor": "",
        "Elevation": "4913.5",
        "RKB": "27.5",
        "Spud Date": "6/4/2024",
        "Days from Spud": "7.67",
        "Days on Loc": "34",
        "MD/TVD": "20537 FT/10719 FT",
        "24 Hr Footage": "3068",
        "Present Operations": "DRILLING LATERAL @ 20,537'.",
        "Activity Planned": "DRILL LATERAL SECTION TO PLANNED TD @ ~21,226', PUMP TD SWEEPS & CHC, SOOH & L/D DRILL PIPE."
    },
    "MUD": {
        "Type": "M

In [0]:
import os
import re
import json
import cv2
import logging
import pandas as pd
import pytesseract
from PIL import Image

# --------------------------------------------------
# Logger configuration (global)
# --------------------------------------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

# --------------------------------------------------
# Helper functions (assumed available)
# --------------------------------------------------
def dbfs_to_local_path(dbfs_path):
    if dbfs_path.startswith("dbfs:/"):
        return "/dbfs/" + dbfs_path[len("dbfs:/"):]
    return dbfs_path

def sanitize_section_name(section):
    return section.lower().replace(" ", "_").replace("/", "_").replace(":", "")

# For brevity, assume safe_read_image_cv2, safe_read_image_pil, preprocess_image,
# perform_ocr, etc. are defined elsewhere and work as before.
# If needed, add logging to these functions too.

# --------------------------------------------------
# MUD Pipeline
# --------------------------------------------------
def process_mud(image_path, debug=True):
    logger.info("[MUD] Starting processing.")
    try:
        img = read_cropped_section_image(image_path)
        logger.debug("[MUD] Image loaded from: %s", image_path)
    except Exception as e:
        logger.error("[MUD] Failed to load image: %s", e)
        raise

    try:
        thresh_img = preprocess_image_mud(img, debug=debug)
        logger.debug("[MUD] Preprocessed image using adaptive thresholding.")
    except Exception as e:
        logger.error("[MUD] Preprocessing failed: %s", e)
        raise

    rois = detect_text_regions_mud(thresh_img, debug=debug)
    logger.info("[MUD] Detected %d text regions.", len(rois))

    roi_texts = perform_ocr_on_rois_mud(img, rois, debug=debug)
    logger.debug("[MUD] Performed OCR on each ROI. Number of ROI texts: %d", len(roi_texts))

    expected_headers = [
        "Type", "Weight In", "Weight Out", "pH", "CAKE",
        "GELS (10s/10m/30m)", "Oil/Water", "FV", "ES", "PV",
        "YP", "CL", "Ca", "LGS", "WL", "HTHP Loss", "3 RPM",
        "6 RPM", "Mud Pits and Hole Volume", "24 Hr Loss",
        "Total Loss", "Comments"
    ]
    mud_dict = build_mud_dict_from_rois(roi_texts, expected_headers)
    if not mud_dict or all(v == "[BLANK]" for v in mud_dict.values() if not isinstance(v, dict)):
        logger.warning("[MUD] Parsed data is empty or all values are blank. Check OCR quality and ROI detection.")
    else:
        logger.info("[MUD] Successfully parsed MUD data.")
    df = pd.DataFrame(list(mud_dict.items()), columns=["Key", "Value"]) if isinstance(mud_dict, dict) else pd.DataFrame(mud_dict)
    logger.info("[MUD] DataFrame shape: %s", df.shape)
    return {"MUD": mud_dict}, df

# --------------------------------------------------
# PERSONNEL Pipeline
# --------------------------------------------------
def process_personnel(image_path, debug=True):
    logger.info("[PERSONNEL] Starting processing.")
    try:
        img = read_cropped_section_image(image_path)
        logger.debug("[PERSONNEL] Image loaded from: %s", image_path)
    except Exception as e:
        logger.error("[PERSONNEL] Failed to load image: %s", e)
        raise

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
    logger.debug("[PERSONNEL] Converted image to grayscale and applied adaptive thresholding.")

    rois = detect_text_regions_personnel(thresh_img, debug=debug)
    logger.info("[PERSONNEL] Detected %d text regions.", len(rois))

    roi_results = perform_ocr_on_rois_personnel(img, rois, debug=debug)
    logger.debug("[PERSONNEL] OCR completed on personnel regions. Number of OCR results: %d", len(roi_results))

    groups = group_rois_by_row(roi_results, y_threshold=20)
    logger.info("[PERSONNEL] Grouped OCR results into %d rows.", len(groups))

    data_dict = preprocess_personnel_data_from_rows(groups)
    if not data_dict.get("PERSONNEL"):
        logger.warning("[PERSONNEL] No personnel data parsed from OCR results.")
    else:
        logger.info("[PERSONNEL] Parsed personnel data successfully.")

    columns = ["Company", "Contractor", "No. Personnel", "Daily Hours", "Cumulative Hours"]
    df = pd.DataFrame(data_dict.get("PERSONNEL", []), columns=columns)
    logger.info("[PERSONNEL] DataFrame shape: %s", df.shape)
    return data_dict, df

# --------------------------------------------------
# DAILY NUMBERS: OBSERVATION & INTERVENTION Pipeline
# --------------------------------------------------
def process_obs_int(image_path, debug=True):
    logger.info("[OBS_INT] Starting processing.")
    try:
        img = safe_read_image_cv2(image_path)
        logger.debug("[OBS_INT] Image loaded from: %s", image_path)
    except Exception as e:
        logger.error("[OBS_INT] Failed to load image: %s", e)
        raise

    thresh = preprocess_image(img, debug=debug)
    logger.debug("[OBS_INT] Image preprocessed for thresholding.")
    rois = detect_text_regions(thresh, debug=debug)
    logger.info("[OBS_INT] Detected %d text regions.", len(rois))
    roi_texts = perform_ocr_on_rois(img, rois, debug=debug)
    logger.debug("[OBS_INT] OCR performed on regions; number of texts: %d", len(roi_texts))

    header_str = "daily numbers: observation & intervention"
    types_list, numbers_list = [], []
    for idx, (_, _, _, _, text) in enumerate(roi_texts):
        clean = text.strip()
        logger.debug("[OBS_INT] Processing ROI %d: '%s'", idx, clean)
        if clean.lower() in [header_str, "number", "[blank]"]:
            logger.debug("[OBS_INT] Skipping header or blank text.")
            continue
        try:
            # If conversion to float succeeds, assume it is a number.
            float(clean)
            numbers_list.append(clean)
            logger.debug("[OBS_INT] Added to numbers: %s", clean)
        except ValueError:
            types_list.append(clean)
            logger.debug("[OBS_INT] Added to types: %s", clean)

    count = max(len(types_list), len(numbers_list))
    while len(types_list) < count:
        types_list.append("")
    while len(numbers_list) < count:
        numbers_list.append("")
    structured = [{"Type": types_list[i], "Number": numbers_list[i]} for i in range(count)]
    logger.info("[OBS_INT] Parsed %d rows for observation & intervention.", count)
    df = pd.DataFrame(structured)
    logger.info("[OBS_INT] DataFrame shape: %s", df.shape)
    return {"DAILY NUMBERS: OBSERVATION & INTERVENTION": structured}, df

# --------------------------------------------------
# Operations Description Parser (within TIME BREAKDOWN)
# --------------------------------------------------
def parse_operations_description(ops_text):
    logger.info("[OPS_DESC] Parsing operations description.")
    ops_data = {
        "Depth": {"From": "", "To": ""},
        "Performance": {"Feet": "", "FPH": ""},
        "Rotation_Slide": {"Rotate": "", "Slide": ""},
        "Rotation_Time": {"Rotate Time": "", "Slide Time": ""},
        "GPM": "",
        "MTR RPM": "",
        "SPP": "",
        "DIFF": "",
        "WOB": "",
        "ROT RPM": "",
        "ON BTM TRQ": "",
        "OFF BTM TRQ": "",
        "GAS": {"Units": "", "Flare": ""},
        "MW": {"In": "", "Out": ""},
        "Targets": [],
        "Observations": []
    }
    depth_match = re.search(r"F/\s*([\d,']+)\s*T/\s*([\d,']+)", ops_text, re.IGNORECASE)
    if depth_match:
        ops_data["Depth"]["From"] = depth_match.group(1)
        ops_data["Depth"]["To"] = depth_match.group(2)
        logger.debug("[OPS_DESC] Parsed Depth: %s", ops_data["Depth"])
    else:
        logger.warning("[OPS_DESC] Depth pattern not found.")

    perf_match = re.search(r"\(([\d,']+)\s*@\s*(\d+)\s*FPH\)", ops_text, re.IGNORECASE)
    if perf_match:
        ops_data["Performance"]["Feet"] = perf_match.group(1)
        ops_data["Performance"]["FPH"] = perf_match.group(2)
        logger.debug("[OPS_DESC] Parsed Performance: %s", ops_data["Performance"])
    else:
        logger.warning("[OPS_DESC] Performance pattern not found.")

    # (Additional regex patterns for Rotation_Slide, Rotation_Time, numeric values, etc.)
    # Log each successful extraction or warn if missing.
    # For brevity, only a couple of examples are shown.
    rs_match = re.search(r"ROTATE\s*([\d.]+%)\s*/\s*SLIDE\s*([\d.]+%)", ops_text, re.IGNORECASE)
    if rs_match:
        ops_data["Rotation_Slide"]["Rotate"] = rs_match.group(1)
        ops_data["Rotation_Slide"]["Slide"] = rs_match.group(2)
        logger.debug("[OPS_DESC] Parsed Rotation_Slide: %s", ops_data["Rotation_Slide"])
    else:
        logger.warning("[OPS_DESC] Rotation_Slide pattern not found.")

    # Assume similar parsing for the remaining fields...
    # For observations and targets:
    segments = re.split(r'(?=\*\*\*)', ops_text)
    obs_list = []
    for seg in segments:
        seg = seg.strip()
        if not seg:
            continue
        if not seg.startswith('***'):
            parts = [p.strip() for p in seg.split('.') if p.strip()]
            obs_list.extend(parts)
        else:
            obs_list.append(seg)
    obs_list = [o.lstrip('* ').strip() for o in obs_list]
    clean_obs = [o for o in obs_list if "TARGET" not in o.upper()]
    targets = [o for o in obs_list if "TARGET" in o.upper()]
    ops_data["Observations"] = clean_obs if clean_obs else ["No observation"]
    ops_data["Targets"] = targets
    logger.info("[OPS_DESC] Parsed observations and targets.")
    return ops_data

# --------------------------------------------------
# TIME BREAKDOWN Pipeline (including operations parsing)
# --------------------------------------------------
def process_time_breakdown(image_paths, debug=True):
    logger.info("[TIME_BREAKDOWN] Starting processing.")
    rows_all = []
    for img_path in image_paths:
        logger.info("[TIME_BREAKDOWN] Processing image: %s", img_path)
        try:
            img = safe_read_image_cv2(img_path)
            logger.debug("[TIME_BREAKDOWN] Image loaded from: %s", img_path)
        except Exception as e:
            logger.error("[TIME_BREAKDOWN] Failed to load image: %s", e)
            continue
        thresh = preprocess_image(img, debug=debug)
        rois = detect_text_regions_tb(thresh, debug=debug)
        ocr_results = perform_ocr_on_rois_tb(img, rois, debug=debug)
        # Assume parse_row_text uses parse_operations_description() internally
        rows = parse_all_rows_from_ocr_groups(ocr_results)
        if not rows:
            full_text = pytesseract.image_to_string(thresh, config="--psm 6")
            rows = parse_all_rows_from_text(full_text)
        logger.info("[TIME_BREAKDOWN] Parsed %d rows from image.", len(rows))
        rows_all.extend(rows)
    final_output = {"TIME BREAKDOWN": rows_all}
    df = pd.json_normalize(rows_all) if rows_all else pd.DataFrame()
    logger.info("[TIME_BREAKDOWN] Final DataFrame shape: %s", df.shape)
    return final_output, df

# --------------------------------------------------
# Aggregator Main Process (unchanged except for added logging)
# --------------------------------------------------
def main():
    logger.info("Starting Aggregator Main Process.")
    debug = True  # or set dynamically

    image_paths = {
        "DAILY DRILLING REPORT": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png",
        "WELL/JOB INFORMATION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_2.png",
        "MUD": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_3.png",
        "SURVEY DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_4.png",
        "DIR INFO": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_5.png",
        "DRILL BITS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_6.png",
        "CASING": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_7.png",
        "BOP": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_8.png",
        "PERSONNEL": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_9.png",
        "DAILY NUMBERS: OBSERVATION & INTERVENTION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_10.png",
        "BHA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_11.png",
        "PUMPS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_12.png",
        "COST DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_13.png",
        "TIME BREAKDOWN": [
            "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png",
            "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"
        ],
        "CONSUMABLES": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"
    }

    output_folder = dbfs_to_local_path("dbfs:/mnt/mini-proj-dd/final_results")
    os.makedirs(output_folder, exist_ok=True)

    aggregated_json = {}
    aggregated_df = pd.DataFrame()

    # List the processes along with the corresponding function and image path.
    processes = [
        ("DAILY DRILLING REPORT", process_daily_drilling_report, image_paths.get("DAILY DRILLING REPORT")),
        ("WELL/JOB INFORMATION", process_well_job_info, image_paths.get("WELL/JOB INFORMATION")),
        ("MUD", process_mud, image_paths.get("MUD")),
        ("SURVEY DATA", process_survey, image_paths.get("SURVEY DATA")),
        ("DIR INFO", process_dir_info, image_paths.get("DIR INFO")),
        ("DRILL BITS", process_drill_bits, image_paths.get("DRILL BITS")),
        ("CASING", process_casing, image_paths.get("CASING")),
        ("BOP", process_bop, image_paths.get("BOP")),
        ("PERSONNEL", process_personnel, image_paths.get("PERSONNEL")),
        ("DAILY NUMBERS: OBSERVATION & INTERVENTION", process_obs_int, image_paths.get("DAILY NUMBERS: OBSERVATION & INTERVENTION")),
        ("BHA", process_bha, image_paths.get("BHA")),
        ("PUMPS", process_pumps, image_paths.get("PUMPS")),
        ("COST DATA", process_cost_data, image_paths.get("COST DATA")),
        ("TIME BREAKDOWN", lambda paths, d: process_time_breakdown(paths, d), image_paths.get("TIME BREAKDOWN")),
        ("CONSUMABLES", process_consumables, image_paths.get("CONSUMABLES"))
    ]

    for section, func, img_path in processes:
        try:
            logger.info("Processing section: %s", section)
            data_json, df = func(img_path, debug)
            safe_section = sanitize_section_name(section)
            aggregated_json[section] = data_json.get(section, data_json)
            aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)
            logger.info("[%s] Output prepared successfully.", section)
            section_json_file = os.path.join(output_folder, f"{safe_section}.json")
            with open(section_json_file, "w") as f:
                json.dump(data_json, f, indent=4)
            section_csv_file = os.path.join(output_folder, f"{safe_section}.csv")
            df.to_csv(section_csv_file, index=False)
            logger.info("[%s] Saved outputs to JSON and CSV.", section)
        except Exception as e:
            logger.error("Section %s processing failed: %s", section, e)

    agg_json_path = os.path.join(output_folder, "aggregated_data.json")
    with open(agg_json_path, "w") as f:
        json.dump(aggregated_json, f, indent=4)
    agg_csv_path = os.path.join(output_folder, "aggregated_data.csv")
    aggregated_df.to_csv(agg_csv_path, index=False)
    logger.info("Aggregated outputs saved to %s and %s.", agg_json_path, agg_csv_path)
    print("----- Aggregated JSON Output -----")
    print(json.dumps(aggregated_json, indent=4))

if __name__ == "__main__":
    main()


[2025-04-05 00:05:10,737] INFO: Starting Aggregator Main Process.
[2025-04-05 00:05:10,737] INFO: Starting Aggregator Main Process.
[2025-04-05 00:05:10,737] INFO: Starting Aggregator Main Process.
INFO:__main__:Starting Aggregator Main Process.
[2025-04-05 00:05:10,744] INFO: Processing section: DAILY DRILLING REPORT
[2025-04-05 00:05:10,744] INFO: Processing section: DAILY DRILLING REPORT
[2025-04-05 00:05:10,744] INFO: Processing section: DAILY DRILLING REPORT
INFO:__main__:Processing section: DAILY DRILLING REPORT
[2025-04-05 00:05:10,745] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
[2025-04-05 00:05:10,745] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
[2025-04-05 00:05:10,745] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
INFO:__main__:Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
[2025-04-05 00

----- Aggregated JSON Output -----
{
    "DAILY DRILLING REPORT": {
        "Report Date": "7/4/2024",
        "Report Num": "11",
        "Rig": "Cyclone 39"
    },
    "WELL/JOB INFORMATION": {
        "Well Name": "Ross Fee 4371-31-7-15 MH",
        "Job Name": "Drilling",
        "Supervisor(s)": "CHAD MILLER / ED COOLEY",
        "Field": "XBE",
        "Sec/Twn/Rng": "31, 43N, 71W",
        "Phone": "307-315-1908",
        "AFE #": "240098",
        "API #": "49-005-78911",
        "Email": "cyclone39@aec-denver.com",
        "Contractor": "",
        "Elevation": "4913.5",
        "RKB": "27.5",
        "Spud Date": "6/4/2024",
        "Days from Spud": "7.67",
        "Days on Loc": "34",
        "MD/TVD": "20537 FT/10719 FT",
        "24 Hr Footage": "3068",
        "Present Operations": "DRILLING LATERAL @ 20,537'.",
        "Activity Planned": "DRILL LATERAL SECTION TO PLANNED TD @ ~21,226', PUMP TD SWEEPS & CHC, SOOH & L/D DRILL PIPE."
    },
    "DIR INFO": [
        {
   

In [0]:
import os
import re
import json
import cv2
import logging
import pandas as pd
import pytesseract
from PIL import Image

# Configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)


# -------------------------------
# Utility Functions
# -------------------------------
def dbfs_to_local_path(dbfs_path):
    """Convert a DBFS URI (e.g. "dbfs:/mnt/xxx") to a local path ("/dbfs/mnt/xxx")."""
    if dbfs_path.startswith("dbfs:/"):
        return "/dbfs/" + dbfs_path[len("dbfs:/"):]
    return dbfs_path

# --------------------------------------------------
# BHA Pipeline (Real Code)
# --------------------------------------------------
def extract_bha_data(image_path):
    logger.info("Starting BHA data extraction.")
    try:
        image = safe_read_image_pil(image_path)
        logger.debug(f"Image successfully loaded from: {image_path}")
    except Exception as e:
        logger.error(f"Error loading image in extract_bha_data: {e}")
        raise

    ocr_text = pytesseract.image_to_string(image)
    logger.debug(f"OCR extraction complete; extracted text length: {len(ocr_text)}")

    patterns = {
        "Drill Pipe Detail": r"Drill Pipe Detail:\s*([^\n]+)",
        "Size": r"Size:\s*([\d.]+)\b",
        "Wt./Ft": r"Wt\./Ft:\s*([\d.]+)\b",
        "Connection": r"Connection:\s*([\w\d-]+)\b",
        "ID": r"ID:\s*([\d.]+)\b",
        "Drill Bit": r"Drill Bit:\s*([^\n;]+)",
        "Motor": r"Motor:\s*([^\n;]+)",
        "MWD Tool": r"MWD Tool:\s*([^\n;]+)",
        "Monel Collar": r"Monel Collar:\s*([^\n;]+)",
        "X-Over": r"X-Over:\s*([^\n;]+)",
        "Sub": r"Sub:\s*([^\n;]+)",
        "HWDP": r"HWDP:\s*([^\n;]+)",
        "Drill Pipe": r"Drill Pipe:\s*([\d.]+(?:\" DP)?)",
        "Reamer": r"Reamer:\s*([^\n;]+)",
        "Shock Sub": r"Shock Sub:\s*([^\n;]+)",
        "Total Length": r"Total Length:\s*(\d+)\b"
    }
    bha_data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, ocr_text)
        if match:
            bha_data[key] = match.group(1).strip()
            logger.debug(f"Extracted {key}: {bha_data[key]}")
        else:
            logger.warning(f"No match found for key: {key}")

    if "Drill Pipe Detail" in bha_data:
        detail = bha_data["Drill Pipe Detail"]
        logger.debug("Cleaning 'Drill Pipe Detail' by removing redundant keys.")
        for remove_key in ["Size", "Wt./Ft", "Connection", "ID"]:
            if remove_key in bha_data:
                detail = re.sub(rf"{remove_key}:\s*{re.escape(bha_data[remove_key])}", "", detail).strip(",; ")
                logger.debug(f"After removing {remove_key}, Drill Pipe Detail: {detail}")
        bha_data["Drill Pipe Detail"] = detail

    structured_data = {
        "BHA": {
            "Drill Pipe Detail": bha_data.get("Drill Pipe Detail", ""),
            "Size": bha_data.get("Size", ""),
            "Wt./Ft": bha_data.get("Wt./Ft", ""),
            "Connection": bha_data.get("Connection", ""),
            "ID": bha_data.get("ID", ""),
            "BHA #4": {
                "Drill Bit": bha_data.get("Drill Bit", ""),
                "Motor": bha_data.get("Motor", ""),
                "MWD Tool": bha_data.get("MWD Tool", ""),
                "Monel Collar": bha_data.get("Monel Collar", ""),
                "X-Over": bha_data.get("X-Over", ""),
                "Sub": bha_data.get("Sub", ""),
                "HWDP": bha_data.get("HWDP", ""),
                "Drill Pipe": bha_data.get("Drill Pipe", ""),
                "Reamer": bha_data.get("Reamer", ""),
                "Shock Sub": bha_data.get("Shock Sub", "")
            },
            "Total Length": bha_data.get("Total Length", "")
        }
    }
    logger.info("BHA data extraction completed.")
    return structured_data

def process_bha(image_path, debug=True):
    logger.info("Processing BHA section.")
    bha_data = extract_bha_data(image_path)
    df = pd.DataFrame([bha_data.get("BHA", {})])
    logger.info(f"BHA DataFrame created with shape: {df.shape}")
    return {"BHA": bha_data.get("BHA", {})}, df

# --------------------------------------------------
# PUMPS Pipeline (Real Code)
# --------------------------------------------------
def parse_pumps_table(ocr_text):
    logger.info("Parsing pumps table from OCR text.")
    pump_pattern = re.compile(
        r"^(\d+)?\s*"
        r"(BOMCO)\s+(TRIPLEX)\s+"
        r"(\d+)?\s*"
        r"(\d+)\s+"
        r"([\d.]+)\s+"
        r"([\d.]+)\s+"
        r"(\d+)\s+"
        r"(\d+)\s+"
        r"(\d+)\s+"
        r"(\d+)\s*$",
        re.IGNORECASE
    )
    pumps = []
    lines = ocr_text.splitlines()
    for idx, line in enumerate(lines):
        line = line.strip()
        match = pump_pattern.match(line)
        if match:
            (number, model, pump_type, hhp, efficiency,
             stroke, liner, p_rating, p_limit,
             spm_rating, spm_limit) = match.groups()
            pump_entry = {
                "Number": number if number else "",
                "Model": model,
                "Type": pump_type,
                "HHP": hhp if hhp else "",
                "Efficiency": efficiency,
                "Stroke(in)": stroke,
                "Liner(in)": liner,
                "P-Rating(psi)": p_rating,
                "P-Limit(psi)": p_limit,
                "SPM Rating": spm_rating,
                "SPM Limit": spm_limit
            }
            pumps.append(pump_entry)
            logger.debug(f"Parsed pump from line {idx}: {pump_entry}")
        else:
            logger.debug(f"No pump match for line {idx}: {line}")
    logger.info(f"Total pumps parsed: {len(pumps)}")
    return pumps

def parse_drilling_circ_rates(ocr_text):
    logger.info("Parsing drilling circulation rates from OCR text.")
    circ_rates = []
    segments = re.split(r"(?=Drilling/Circ Rate \d+)", ocr_text)
    pattern = re.compile(
        r"Drilling/Circ Rate\s+(\d+).*?"
        r"(\d+)\s+PS[!I].*?"
        r"@\s*(\d+).*?"
        r"([\d.]+)\s+Gal/Stoke.*?"
        r"([\d.]+)\s+GPM.*?"
        r"([\d.]+)\s+BPM.*?"
        r"([\d.]+)\s+DC.*?"
        r"([\d.]+)\s+DP",
        re.IGNORECASE | re.DOTALL
    )
    for seg in segments:
        seg = seg.strip()
        if not seg.startswith("Drilling/Circ Rate"):
            continue
        seg_clean = " ".join(seg.splitlines())
        match = pattern.search(seg_clean)
        if match:
            rate_id, pressure, spm, gal_stroke, gpm, bpm, dc, dp = match.groups()
            circ_rate_entry = {
                "RateID": rate_id,
                "Pressure(PSI)": pressure,
                "SPM": spm,
                "Gal/Stoke": gal_stroke,
                "GPM": gpm,
                "BPM": bpm,
                "DC": dc,
                "DP": dp
            }
            circ_rates.append(circ_rate_entry)
            logger.debug(f"Parsed drilling circ rate: {circ_rate_entry}")
        else:
            logger.warning(f"No match in segment: {seg_clean}")
    logger.info(f"Total drilling circ rates parsed: {len(circ_rates)}")
    return circ_rates

def process_pumps(image_path, debug=True):
    logger.info("Processing PUMPS section.")
    try:
        pil_img = safe_read_image_pil(image_path)
        logger.debug(f"Image successfully loaded for pumps from: {image_path}")
    except Exception as e:
        logger.error(f"Error loading pumps image: {e}")
        raise

    ocr_text = pytesseract.image_to_string(pil_img, config="--psm 6").strip()
    logger.debug(f"Pumps OCR extraction complete; text length: {len(ocr_text)}")
    pumps = parse_pumps_table(ocr_text)
    circ_rates = parse_drilling_circ_rates(ocr_text)
    final_data = {"Pumps": pumps, "DrillingCircRates": circ_rates}

    df_pumps = pd.DataFrame(pumps)
    df_circ = pd.DataFrame(circ_rates)
    if not df_pumps.empty and not df_circ.empty:
        df = pd.concat([df_pumps, df_circ], axis=0, ignore_index=True)
    elif not df_pumps.empty:
        df = df_pumps
    else:
        df = df_circ
    logger.info(f"PUMPS processing completed; combined DataFrame shape: {df.shape}")
    return {"PUMPS": final_data}, df

# --------------------------------------------------
# COST DATA Pipeline (Real Code)
# --------------------------------------------------
def process_cost_data(img_path, debug=True):
    logger.info("Processing COST DATA section.")
    try:
        img = safe_read_image(img_path)
        logger.debug(f"Image loaded for COST DATA from: {img_path}")
    except Exception as e:
        logger.error(f"Error loading COST DATA image: {e}")
        raise

    ocr_text = perform_ocr(img)
    logger.debug(f"COST DATA OCR extraction complete; text length: {len(ocr_text)}")
    expected_keys = [
        "Drilling AFE Amount", "Daily Drilling Cost", "Cumulative Drilling Cost",
        "Cumulative Well Cost", "Daily Mud Cost", "Cumulative Mud Cost"
    ]
    extracted = extract_key_value_from_text_cost(ocr_text, expected_keys)
    df = pd.DataFrame(list(extracted.items()), columns=["Key", "Value"])
    logger.info(f"COST DATA processing completed; DataFrame shape: {df.shape}")
    return {"COST DATA": extracted}, df

# --------------------------------------------------
# TIME BREAKDOWN Pipeline (Real Code)
# --------------------------------------------------
def detect_text_regions_tb(thresh_img, debug=True):
    logger.debug("Detecting text regions for TIME BREAKDOWN.")
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rois = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 30 and h > 15:
            rois.append((x, y, w, h))
    rois.sort(key=lambda b: (b[1], b[0]))
    logger.debug(f"Detected {len(rois)} text regions for TIME BREAKDOWN.")
    return rois

def perform_ocr_on_rois_tb(img, rois, debug=True):
    logger.debug("Performing OCR on regions for TIME BREAKDOWN.")
    results = []
    for (x, y, w, h) in rois:
        roi = img[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi, config="--psm 6").strip() or "[BLANK]"
        results.append((x, y, w, h, text))
        logger.debug(f"TIME BREAKDOWN OCR Box ({x},{y},{w},{h}): {text}")
    return results

def group_ocr_rows(roi_results, y_threshold=20):
    logger.debug("Grouping OCR results into rows for TIME BREAKDOWN.")
    groups = []
    roi_results_sorted = sorted(roi_results, key=lambda r: r[1])
    current_group = []
    current_y = None
    for roi in roi_results_sorted:
        x, y, w, h, text = roi
        if current_y is None or abs(y - current_y) <= y_threshold:
            current_group.append(roi)
            current_y = y if current_y is None else (current_y + y) / 2
        else:
            groups.append(current_group)
            current_group = [roi]
            current_y = y
    if current_group:
        groups.append(current_group)
    logger.debug(f"Grouped into {len(groups)} rows for TIME BREAKDOWN.")
    return groups

def parse_all_rows_from_ocr_groups(roi_results):
    logger.debug("Parsing rows from grouped OCR results for TIME BREAKDOWN.")
    rows = []
    groups = group_ocr_rows(roi_results, y_threshold=20)
    for group in groups:
        group_sorted = sorted(group, key=lambda r: r[0])
        row_text = " ".join([text for (x, y, w, h, text) in group_sorted])
        parsed = parse_row_text(row_text)  # Assumes parse_row_text is defined elsewhere
        if parsed:
            rows.append(parsed)
            logger.debug(f"Parsed row: {parsed}")
    logger.info(f"Parsed a total of {len(rows)} rows from TIME BREAKDOWN OCR groups.")
    return rows

def parse_all_rows_from_text(full_text):
    logger.debug("Parsing rows directly from full OCR text for TIME BREAKDOWN.")
    row_chunks = re.split(r"(?=\d{2}:\d{2}\s+\d{2}:\d{2})", full_text)
    rows = []
    for chunk in row_chunks:
        chunk = chunk.strip()
        parsed = parse_row_text(chunk)  # Assumes parse_row_text is defined elsewhere
        if parsed:
            rows.append(parsed)
            logger.debug(f"Parsed row from text: {parsed}")
    logger.info(f"Parsed {len(rows)} rows from full OCR text for TIME BREAKDOWN.")
    return rows

def process_time_breakdown(image_paths, debug=True):
    logger.info("Processing TIME BREAKDOWN section.")
    rows_all = []
    for img_path in image_paths:
        logger.info(f"Processing TIME BREAKDOWN image: {img_path}")
        try:
            img = safe_read_image_cv2(img_path)
            logger.debug(f"Image loaded for TIME BREAKDOWN from: {img_path}")
        except Exception as e:
            logger.error(f"Error loading TIME BREAKDOWN image {img_path}: {e}")
            continue
        thresh = preprocess_image(img, debug=debug)  # Assumes preprocess_image is defined elsewhere
        rois = detect_text_regions_tb(thresh, debug=debug)
        ocr_results = perform_ocr_on_rois_tb(img, rois, debug=debug)
        rows = parse_all_rows_from_ocr_groups(ocr_results)
        if not rows:
            full_text = pytesseract.image_to_string(thresh, config="--psm 6")
            rows = parse_all_rows_from_text(full_text)
        rows_all.extend(rows)
    final_output = {"TIME BREAKDOWN": rows_all}
    df = pd.json_normalize(rows_all) if rows_all else pd.DataFrame()
    logger.info(f"TIME BREAKDOWN processing completed; total rows parsed: {len(rows_all)}")
    return final_output, df

# --------------------------------------------------
# CONSUMABLES Pipeline (Real Code)
# --------------------------------------------------
def build_consumables_dict_from_rois(roi_texts, debug=False):
    logger.info("Building consumables dictionary from OCR regions.")
    row_tolerance = 10
    rows = []
    current_row = []
    prev_y = None
    for (x, y, w, h, text) in roi_texts:
        if prev_y is None or abs(y - prev_y) <= row_tolerance:
            current_row.append((x, y, w, h, text))
        else:
            rows.append(current_row)
            current_row = [(x, y, w, h, text)]
        prev_y = y
    if current_row:
        rows.append(current_row)

    grouped_rows = []
    for i, row in enumerate(rows):
        row.sort(key=lambda cell: cell[0])
        line = " ".join(cell[4] for cell in row).strip()
        grouped_rows.append(line)
        logger.info(f"Consumables Row {i}: {line}")

    data_rows = []
    for line in grouped_rows:
        lower_line = line.lower()
        if ("consumable" in lower_line and "received" in lower_line) or "nun" in lower_line:
            continue
        if len(line.split()) < 5:
            continue
        data_rows.append(line)

    consumables_list = []
    for line in data_rows:
        tokens = re.split(r'\s+', line)
        if len(tokens) > 5:
            first = " ".join(tokens[:-4])
            tokens = [first] + tokens[-4:]
        if len(tokens) != 5:
            logger.warning(f"Skipping row due to unexpected token count: {tokens}")
            continue
        row_dict = {
            "Consumable": tokens[0],
            "Daily Received (gal)": tokens[1],
            "Daily Used (gal)": tokens[2],
            "Cumulative Used (gal)": tokens[3],
            "Daily on Hand (gal)": tokens[4]
        }
        consumables_list.append(row_dict)
        logger.debug(f"Parsed consumables row: {row_dict}")

    logger.info("Completed building consumables dictionary.")
    return consumables_list

def process_consumables(image_path, debug=True):
    logger.info("Processing CONSUMABLES section.")
    try:
        img = safe_read_image_cv2(image_path)
        logger.debug(f"Image successfully loaded for CONSUMABLES from: {image_path}")
    except Exception as e:
        logger.error(f"Error loading CONSUMABLES image: {e}")
        raise
    thresh = preprocess_image(img, debug=debug)  # Assumes preprocess_image is defined elsewhere
    rois = detect_text_regions(thresh, debug=debug)  # Assumes detect_text_regions is defined elsewhere
    logger.info(f"Detected {len(rois)} text regions for CONSUMABLES.")
    roi_texts = perform_ocr_on_rois(img, rois, debug=debug)  # Assumes perform_ocr_on_rois is defined elsewhere
    consumables_list = build_consumables_dict_from_rois(roi_texts, debug=debug)
    df = pd.DataFrame(consumables_list)
    logger.info(f"CONSUMABLES processing completed; DataFrame shape: {df.shape}")
    return {"CONSUMABLES": consumables_list}, df

# --------------------------------------------------
# Aggregator Main Process
# --------------------------------------------------
def main():
    logger.info("Starting Aggregator Main Process.")
    debug = True  # Enable verbose logging
    
    # Define image paths for all sections.
    image_paths = {
        "DAILY DRILLING REPORT": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png",
        "WELL/JOB INFORMATION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_2.png",
        "MUD": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_3.png",
        "SURVEY DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_4.png",
        "DIR INFO": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_5.png",
        "DRILL BITS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_6.png",
        "CASING": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_7.png",
        "BOP": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_8.png",
        "PERSONNEL": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_9.png",
        "DAILY NUMBERS: OBSERVATION & INTERVENTION": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_10.png",
        "BHA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_11.png",
        "PUMPS": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_12.png",
        "COST DATA": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_13.png",
        "TIME BREAKDOWN": [
            "dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_14.png",
            "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"
        ],
        "CONSUMABLES": "dbfs:/mnt/mini-proj-dd/cropped_sections/page_2_section_2.png"
    }
    
    output_folder = dbfs_to_local_path("dbfs:/mnt/mini-proj-dd/final_results")
    os.makedirs(output_folder, exist_ok=True)
    
    aggregated_json = {}
    aggregated_df = pd.DataFrame()
    
    processes = [
        ("DAILY DRILLING REPORT", process_daily_drilling_report, image_paths.get("DAILY DRILLING REPORT")),
        ("WELL/JOB INFORMATION", process_well_job_info, image_paths.get("WELL/JOB INFORMATION")),
        ("MUD", process_mud, image_paths.get("MUD")),
        ("SURVEY DATA", process_survey, image_paths.get("SURVEY DATA")),
        ("DIR INFO", process_dir_info, image_paths.get("DIR INFO")),
        ("DRILL BITS", process_drill_bits, image_paths.get("DRILL BITS")),
        ("CASING", process_casing, image_paths.get("CASING")),
        ("BOP", process_bop, image_paths.get("BOP")),
        ("PERSONNEL", process_personnel, image_paths.get("PERSONNEL")),
        ("DAILY NUMBERS: OBSERVATION & INTERVENTION", process_obs_int, image_paths.get("DAILY NUMBERS: OBSERVATION & INTERVENTION")),
        ("BHA", process_bha, image_paths.get("BHA")),
        ("PUMPS", process_pumps, image_paths.get("PUMPS")),
        ("COST DATA", process_cost_data, image_paths.get("COST DATA")),
        ("TIME BREAKDOWN", lambda paths, d: process_time_breakdown(paths, d), image_paths.get("TIME BREAKDOWN")),
        ("CONSUMABLES", process_consumables, image_paths.get("CONSUMABLES"))
    ]
    
    for section, func, img_path in processes:
        try:
            logger.info(f"Processing section: {section}")
            data_json, df = func(img_path, debug)
            safe_section = sanitize_section_name(section)
            aggregated_json[section] = data_json.get(section, data_json)
            aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)
            logger.info(f"{section} output prepared successfully.")
            section_json_file = os.path.join(output_folder, f"{safe_section}.json")
            with open(section_json_file, "w") as f:
                json.dump(data_json, f, indent=4)
            section_csv_file = os.path.join(output_folder, f"{safe_section}.csv")
            df.to_csv(section_csv_file, index=False)
            logger.info(f"Saved {section} outputs to JSON and CSV.")
        except Exception as e:
            logger.error(f"{section} processing failed: {e}")
    
    agg_json_path = os.path.join(output_folder, "aggregated_data.json")
    with open(agg_json_path, "w") as f:
        json.dump(aggregated_json, f, indent=4)
    agg_csv_path = os.path.join(output_folder, "aggregated_data.csv")
    aggregated_df.to_csv(agg_csv_path, index=False)
    logger.info(f"Aggregated outputs saved to {agg_json_path} and {agg_csv_path}.")
    print("----- Aggregated JSON Output -----")
    print(json.dumps(aggregated_json, indent=4))

if __name__ == "__main__":
    main()


[2025-04-05 00:06:01,264] INFO: Starting Aggregator Main Process.
[2025-04-05 00:06:01,264] INFO: Starting Aggregator Main Process.
[2025-04-05 00:06:01,264] INFO: Starting Aggregator Main Process.
[2025-04-05 00:06:01,264] INFO: Starting Aggregator Main Process.
INFO:__main__:Starting Aggregator Main Process.
[2025-04-05 00:06:01,268] INFO: Processing section: DAILY DRILLING REPORT
[2025-04-05 00:06:01,268] INFO: Processing section: DAILY DRILLING REPORT
[2025-04-05 00:06:01,268] INFO: Processing section: DAILY DRILLING REPORT
[2025-04-05 00:06:01,268] INFO: Processing section: DAILY DRILLING REPORT
INFO:__main__:Processing section: DAILY DRILLING REPORT
[2025-04-05 00:06:01,275] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
[2025-04-05 00:06:01,275] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sections/page_1_section_1.png
[2025-04-05 00:06:01,275] INFO: Processing DDR image from: dbfs:/mnt/mini-proj-dd/cropped_sectio

----- Aggregated JSON Output -----
{
    "DAILY DRILLING REPORT": {
        "Report Date": "7/4/2024",
        "Report Num": "11",
        "Rig": "Cyclone 39"
    },
    "WELL/JOB INFORMATION": {
        "Well Name": "Ross Fee 4371-31-7-15 MH",
        "Job Name": "Drilling",
        "Supervisor(s)": "CHAD MILLER / ED COOLEY",
        "Field": "XBE",
        "Sec/Twn/Rng": "31, 43N, 71W",
        "Phone": "307-315-1908",
        "AFE #": "240098",
        "API #": "49-005-78911",
        "Email": "cyclone39@aec-denver.com",
        "Contractor": "",
        "Elevation": "4913.5",
        "RKB": "27.5",
        "Spud Date": "6/4/2024",
        "Days from Spud": "7.67",
        "Days on Loc": "34",
        "MD/TVD": "20537 FT/10719 FT",
        "24 Hr Footage": "3068",
        "Present Operations": "DRILLING LATERAL @ 20,537'.",
        "Activity Planned": "DRILL LATERAL SECTION TO PLANNED TD @ ~21,226', PUMP TD SWEEPS & CHC, SOOH & L/D DRILL PIPE."
    },
    "DIR INFO": [
        {
   